<a href="https://colab.research.google.com/github/thuc-github/MIS710-T12023/blob/main/Week%203/MIS710_Lab3-Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MIS710 Lab 3 Week 3**
Authors: Associate Professor Lemai Nguyen and Thuc Nguyen

Objective: to learn and practise linear regression models with scikit-learn


**To do before the class:**
1. complete Labs 0, 1 and 2
2. learn Lecture 3: Supervised Machine Learning: Linear Regression
3. download the housing.csv dataset and store it in your Google drive, Colab folder, MIS710 folder. This is for Task 1.

**Student name:**

Student ID:

# **Task 1: House Price Prediction**

Dataset: HousingPrice

Source: https://www.kaggle.com/datasets/yasserh/housing-prices-dataset

The dataset was modified to allow you deal with missing data.
**Download the modified data at the unit site.**

## **1.1 Importing Libraries**


In [ ]:
#load libraries
import pandas as pd #for data manipulation and analysis
import numpy as np #for working with arrays

#import data visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns



## **1.2 Mount your Google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **1.3 Load data**


1.   Load the dataset
2.   Inspect the data



In [ ]:
# load dataset
records = pd.read_csv("Housing.csv")

#explore the dataset
print(records)

print('Sample size:', records.shape[0])
print('Number of columns:', records.shape[1])

In [ ]:
print(records.info())
print(records.shape)

In [ ]:
#area is wrongly documented as string, convert it to numeric
records['area'] = pd.to_numeric(records['area'], errors='coerce')

## **1.4 Inspect missing data**

The data preprocessing is cyclic with analysing and visualising data, handling missing data, and feature engineering. For the learning purpose, we show you some simple techniques, you should move between the activities yourself.

In [ ]:
#learn to use for loop, and accessing elements of a dataframe using iloc
#Count missing data
for i in records.iloc[:,0:]:
  miss=records[i].isna().sum()
  print(i,'missing: ', miss)

In [ ]:
#another way to find missing data is using the function isnull()
#read about isnull() here https://pandas.pydata.org/docs/reference/api/pandas.isnull.html
#read further at https://www.sharpsightlabs.com/blog/pandas-isnull/
print(records.isnull().sum().sort_values(ascending=0))

## **1.5 Variable analysis**

Stats and visualtion
1.  Univariate analysis
2.  Bivariate analysis
3.  Multivariate analysis




### **1.5.1 Univariate analysis**
Explore and visualise each variable at a time


In [ ]:
#overview of numeric data
records.describe()


In [ ]:
#set the formatting for floating numbers
pd.set_option('display.float_format', lambda x: '%.3f' % x)

data_types =['object', 'float', 'int']
records.describe(include=data_types)

**Use stats results to decide on and handle missing data**

In [ ]:
#describe categorical variables
records['area'].describe()

In [ ]:
sns.histplot(data=records, x='area')

In [ ]:
sns.boxplot(data=records, x='area', showmeans=True)

Area is skewed so replace missing data with median. Let's do it later.

Write your own code to analyse price and generate histogram and boxplot for price

In [ ]:
#Write your code below to explore the variable price


In [ ]:
#describe categorical variables
sns.countplot(data=records, x='furnishingstatus')

Mode is semi-furnished; can you see the space ' '? is it valid?

In [ ]:
#you can also use the built=in function mode; it returns a series, so get the first index, there can be more than one mode

print(records['furnishingstatus'].mode()[0])


In [ ]:
#write your code to visualise other categorical variables, e.g. mainroad


In [ ]:
#write code to display mode of mainroad


**Visualise numerical variables using a for loop**

In [ ]:
#using seaborn https://seaborn.pydata.org/generated/seaborn.histplot.html
# Get the numerical variables from the dataset
numerical_variables = records.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Print the list of numerical variables
print("Numerical Variables:", numerical_variables)

# Display histograms using seaborn
for variable in numerical_variables:
    sns.histplot(data=records, x=variable)
    plt.title(f"Histogram of {variable}")
    plt.show()



In [ ]:
# Create boxplots using seaborn
sns.boxplot(data=records, x='area', showmeans=True)
plt.title(f"Boxplot of area")
plt.show()


In [ ]:
#It's your turn: display the boxplot of price



**Explore each categorical variable**

In [ ]:
#explore each categorial variable
print(records['furnishingstatus'].value_counts())


In [ ]:
#find mode
print('Furnishing Status mode: ', records['furnishingstatus'].mode())

Do it yourself for other categorical variables

**Visualise each categorical variable**

In [ ]:
#explore each categorial variable
print(records['bathrooms'].value_counts())

In [ ]:
#Using seaborn
sns.countplot(x=records['bathrooms'])

In [ ]:
#write your code to visualise other categorical variables, one at a time


**Visualise multiple categorical variables**

In [ ]:
cat_variables = ['bedrooms', 'bathrooms','stories', 'parking','mainroad','guestroom','basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']
for i in cat_variables:
   plt.figure()
   sns.countplot(x=records[i])


Note spaces are included in some catagorical variables. They should be treated as missing data.

### **1.5.3 Treat missing data**

In [ ]:
# Replace empty strings (" ") with NaN
records.replace(" ", pd.NA, inplace=True)

In [ ]:
#Write your code to review missing data, note the changes; now there are missing data, why?


In [ ]:
#Fill in missing numerical data with mean and categorical data with mode
records['area'].fillna(records['area'].median(),inplace=True)
records['furnishingstatus'].fillna(records['furnishingstatus'].mode()[0], inplace=True) #note, there can be more than one mode

#do it yourself for other missing data





In [ ]:
print(records.isnull().sum().sort_values(ascending=0))

### **1.5.2 Multivariate visualisation**

**Display a countplot for one categorical variable grouped by a second categorical variable**
https://seaborn.pydata.org/generated/seaborn.countplot.html

In [ ]:
sns.countplot(data=records, x='prefarea', hue='mainroad')

In [ ]:
#Do it yourself for other categorical variables



**Compare distributions of numerical variables using boxplots**
https://seaborn.pydata.org/generated/seaborn.boxplot.html

In [ ]:
sns.boxplot(data=records, x='price', y='mainroad')

**Encoding categorical data**

In [ ]:
#convert categorical variables to numeric
# Define the custom mapping
furnishing_mapping = {
    'furnished': 2,
    'semi-furnished': 1,
    'unfurnished': 0
}
# Convert the categories to numerical values using replace()
records['furnishingstatus_N'] = records['furnishingstatus'].replace(furnishing_mapping)

In [ ]:
records

In [ ]:
other_cat_variables = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Convert categorical variables into numeric using dummy encoding
records = pd.get_dummies(records, columns=other_cat_variables, drop_first=True)

In [ ]:
#Write code to examine the dataset info()
records

In [ ]:
#Do it yourself for other variables, hint: using x=  y= and hue=


**Ploting diagram to see relationships between two numerical variables**
https://seaborn.pydata.org/generated/seaborn.scatterplot.html

In [ ]:
sns.scatterplot(data=records, x='area', y='price')

In [ ]:
#generate heatmaps to explore relationships
sns.heatmap(records[numerical_variables].corr(), square=True, cmap='Blues', annot=True)
plt.show()

In [ ]:
#generate dendrograms to show hierarchical clustering
sns.clustermap(records[numerical_variables].corr(), square=True, cmap='Blues', annot=True, row_cluster=False)
plt.show()

## **1.6 Optional exercise: Practise accessing columns and rows**

In [ ]:
#Let't move price to the first column
first_column=records.pop('price')
records.insert(0,'price',first_column)

In [ ]:
records

In [ ]:
records.iloc[9:14]

## **1.7 Practise: Encoding data (Optional)**

In [ ]:
#Last week, we learned to convert categorical variables to numerical using LabelEncoder
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
records['mainroad_N'] = encoder.fit_transform(records['mainroad'])
records['basement_N'] = encoder.fit_transform(records['basement'])


In [ ]:
#there are other ways of doing this, for example
records['hotwaterheating_N'] = records['hotwaterheating'].apply(lambda x: 1 if x == 'yes' else 0)

records.sample(10)

In [ ]:
#Another way is getting all catagorical columns
cat_variables = records.select_dtypes(include=['object']).columns
#Convert categorical columns to numeric
records[cat_variables] = records[cat_variables].apply(encoder.fit_transform)

# Display the updated dataset
print(records)

In [ ]:
#OPTIONAL
#another day, defining your OWN function
#convert categorical data to numerical
def coding_furnishingstatus(x):
        if x=='furnished': return 3
        if x=='semi-furnished': return 2
        if x=='unfurnished': return 1

records['furnishingstatus_N'] = records['furnishingstatus'].apply(coding_furnishingstatus)

records.iloc[9:14]

In [ ]:
#write code to drop redudant columns
records= records.drop(['mainroad_N','basement_N','hotwaterheating_N','furnishingstatus_N'], axis=1)
print(records.info())

**Let's go back to the previous way**

In [ ]:
# Define the custom mapping
furnishing_mapping = {
    'furnished': 2,
    'semi-furnished': 1,
    'unfurnished': 0
}
# Convert the categories to numerical values using replace()
records['furnishingstatus_N'] = records['furnishingstatus'].replace(furnishing_mapping)

In [ ]:
records[['furnishingstatus', 'furnishingstatus_N']]

In [ ]:
other_cat_variables = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Convert categorical variables into numeric using dummy encoding
records = pd.get_dummies(records, columns=other_cat_variables, drop_first=True)

In [ ]:
#Write code to examine the dataset info()


## **1.8 Feature selection**

In [ ]:
#feature selection
features=['area']
X=records[features]
X.head()

In [ ]:
#specify the label
y=records['price']
y.head()

## **1.9 Split the dataset**

Split arrays or matrices into random train and test subsets
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train_test_split#sklearn.model_selection.train_test_split

In [ ]:
from sklearn.model_selection import train_test_split # Import train_test_split function

# Split dataset into training set 80% and test set 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2023)

#inspect the split datasets
print(X_train.head())
print(y_train.head())

print('Training dataset size:',X_train.shape)
print('Test dataset size:',X_test.shape)


## **1.10 Training a Linear Regression Model**

1.   Train a model using the training dataset
2.   Make prediction using the model for the test dataset

Read about Linear Regression https://scikit-learn.org/stable/modules/linear_model.html

LinearRegression will take in its fit method arrays X, y and will store the coefficients of the linear model in its coef_ member






In [ ]:
#import linear_model
from sklearn import linear_model

#create a linear_model object
reg = linear_model.LinearRegression()

**Train a model**

In [ ]:
# Train a Regression model (regressor) with the training dataset
reg=reg.fit(X_train, y_train)

**Make predictions using the model and the test set**

In [ ]:
#Make predictions for the test dataset
y_pred = reg.predict(X_test)


**Inspect the predictions and the original labels**

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("Actual prices")
plt.ylabel("Predicted prices")
plt.title("Actual prices vs Predicted prices")
plt.show()

In [ ]:
#set the formatting for floating numbers
pd.set_option('display.float_format', lambda x: '%.0f' % x)
area=X_test['area']
#inspection
inspection=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
inspection=pd.DataFrame({'Area':area, 'Actual':y_test, 'Predicted':y_pred})
inspection.head(20)

**Getting the Intercept and Coefficients**

In [ ]:
print('%.2f' % reg.intercept_)
print('%.2f' % reg.coef_)
print('Price = ', '%.2f' % reg.intercept_, ' + ', '%.2f' % reg.coef_, ' * ', 'Area' )


In [ ]:
sns.scatterplot(data=inspection, x='Area', y='Actual')
sns.regplot(data=inspection, x='Area', y='Predicted', color='blue')

## **1.11 Performance evaluation**

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2:.2f}")

In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate and print the mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.0f}")

In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate and print the root mean square error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Square Error: {rmse:.0f}")

In [ ]:
#Examine the performance using the descriptive stats of price
inspection['Actual'].describe()

In [ ]:
#Examine the performance using the descriptive stats of predicted price
inspection['Predicted'].describe()

## **1.12 Multi-linear regression**
Repeat from the previous steps to train and test a multiple linear regression model

### **1.12.1 Select features**

In [ ]:
#Examine the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(records[numerical_variables].corr(), annot=True, cmap='Blues')
plt.title("Correlation Matrix")
plt.show()

In [ ]:
#write code to show info()

In [ ]:
#select your own features and train and evaluate a model. For example
features=['area','bedrooms', 'bathrooms','stories', 'parking','mainroad_yes','guestroom_yes','basement_yes', 'hotwaterheating_yes', 'airconditioning_yes', 'prefarea_yes', 'furnishingstatus_N']
X=records[features]

In [ ]:
#specify the label y=



### **1.12.2 Split data**

In [ ]:
# Split dataset into training set 80% and test set 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2023)

#inspect the split datasets
print(X_train.head())
print(y_train.head())

print('Training dataset size:',X_train.shape)
print('Test dataset size:',X_test.shape)


### **1.12.3. Train a linear model**

In [ ]:
#create a linear_model object, hint reg=


In [ ]:
# Train a Regression model (regressor) with the training dataset, hint: fit(X_train, y_train)



### **1.12.4 Make predictions**
Make predictions using the model and the test set

In [ ]:
#Make predictions for the test dataset, hint y_pred=



**Inspect the predictions and the original labels**

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("Actual prices")
plt.ylabel("Predicted prices")
plt.title("Actual prices vs Predicted prices")
plt.show()

In [ ]:
#set the formatting for floating numbers
pd.set_option('display.float_format', lambda x: '%.0f' % x)
area=X_test['area']
#inspection
inspection=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
inspection=pd.DataFrame({'Area':area, 'Actual':y_test, 'Predicted':y_pred})
inspection.head(20)

### **1.12.5 Performance evaluation**

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2:.2f}")

In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate and print the mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.0f}")

In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate and print the root mean square error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Square Error: {rmse:.0f}")

In [ ]:
#Examine the performance using the descriptive stats of price
inspection['Actual'].describe()

In [ ]:
#Examine the performance using the descriptive stats of predicted price
inspection['Predicted'].describe()

# **Task 2: Insurance cost estimation**

**Do it yourself:** Repeat the above steps with the insurance dataset to consilidate your learning

## **2.1 Import libraries**

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error

## **2.2 Load dataset**

In [ ]:
# Load data using pandas.read_csv(filepath_or_url, sep=',')
url = 'https://raw.githubusercontent.com/thuc-github/MIS710-T12023/main/Week%203/insurance.csv'

df = pd.read_csv(url)


## **2.3 EDA**

* How many rows and columns in the dataset?
* Return the first n rows.
* What are the columns and their datatypes?
* Is there any missing values?
* How to deal with categorical features?
* Any strong correlation from the dataset?  
* What are the stats for the `charges`? Plot overall distribution of `charges`; and ditribution of chareges for smoker and non-smokers. Practice more with `bmi`, `age` and `sex` variables.



In [ ]:
# How many rows and columns in the dataset?
df

# Return the first n rows.
df.head()

# What are the columns and their datatypes?
df.info()

# Is there any missing values?
df.isnull().sum()

# Any strong correlation from the dataset?
df.corr()

In [ ]:
# Correlation plot
f, ax = plt.subplots(figsize=(10, 8))
corr = df.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(240,10,as_cmap=True),
            square=True, ax=ax)

In [ ]:
# How to deal with categorical features?

from sklearn.preprocessing import LabelEncoder
#sex
le = LabelEncoder()
le.fit(df.sex.drop_duplicates())
df.sex = le.transform(df.sex)
# smoker or not
le.fit(df.smoker.drop_duplicates())
df.smoker = le.transform(df.smoker)
#region
le.fit(df.region.drop_duplicates())
df.region = le.transform(df.region)


In [ ]:
'''
What are the stats for the charges? Plot overall distribution of charges;
and ditribution of chareges for smoker and non-smokers.
'''
df.charges.describe()

In [ ]:
df.charges.hist(bins=50, figsize=(12,8))

In [ ]:
df.charges.hist(by=df.smoker, bins=50, figsize=(12,8))

In [ ]:
# Alternative using seaborn

f= plt.figure(figsize=(12,5))

ax=f.add_subplot(121)
sns.histplot(df[(df.smoker == 1)]["charges"],color='c',ax=ax)
ax.set_title('Distribution of charges for smokers')

ax=f.add_subplot(122, sharex = ax)
sns.histplot(df[(df.smoker == 0)]['charges'],color='b',ax=ax)
ax.set_title('Distribution of charges for non-smokers')

## **2.4 Data preparation**


1.   Prepare X, y
2.   Prepare X_train, X_test, y_train, y_test (hint: using `train_test_split')



In [ ]:
X = df.drop(['charges'], axis = 1)
y = df.charges

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

## **2.5 Model implementation**

1. Try with the original data. What's the performance?
2. Let's add data normalisation. Has the performance been improved?

In [ ]:
lr = LinearRegression().fit(X_train,y_train)

y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

print('MSE_Train: {}, MSE_Test: {}, MAE_Train: {}, MAE_Test: {}'.format(mean_squared_error(y_train, y_train_pred),
                                                      mean_squared_error(y_test, y_test_pred),
                                                      mean_absolute_error(y_train, y_train_pred),
                                                      mean_absolute_error(y_test, y_test_pred)))

print('R2 train data: %.3f, R2 test data: %.3f' % (
r2_score(y_train,y_train_pred),
r2_score(y_test,y_test_pred)))

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(y_train_pred, y_train_pred - y_train,
          c = 'black', marker = 'o', s = 35, alpha = 0.5,
          label = 'Train data')
plt.scatter(y_test_pred, y_test_pred - y_test,
          c = 'c', marker = 'o', s = 35, alpha = 0.7,
          label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Tailings')
plt.legend(loc = 'upper left')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'red')
plt.show()

In [ ]:
quad = PolynomialFeatures(degree = 2)
X_quad = quad.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_quad, y, test_size=0.2, random_state = 0)

plr = LinearRegression().fit(X_train, y_train)

y_train_pred = plr.predict(X_train)
y_test_pred = plr.predict(X_test)

print('MSE_Train: {}, MSE_Test: {}, MAE_Train: {}, MAE_Test: {}'.format(mean_squared_error(y_train, y_train_pred),
                                                      mean_squared_error(y_test, y_test_pred),
                                                      mean_absolute_error(y_train, y_train_pred),
                                                      mean_absolute_error(y_test, y_test_pred)))

print('R2 train data: %.3f, R2 test data: %.3f' % (
r2_score(y_train,y_train_pred),
r2_score(y_test,y_test_pred)))

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(y_train_pred, y_train_pred - y_train,
          c = 'black', marker = 'o', s = 35, alpha = 0.5,
          label = 'Train data')
plt.scatter(y_test_pred, y_test_pred - y_test,
          c = 'c', marker = 'o', s = 35, alpha = 0.7,
          label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Tailings')
plt.legend(loc = 'upper left')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'red')
plt.show()